# Midi to corpus (not flatten)

In [1]:
from tqdm import tqdm
import random
import pretty_midi
import muspy
import mido
import pandas as pd
import os
import numpy as np
import collections
import pickle
import json

from midi2corpus import *

In [2]:
index = pd.read_csv('../../MusicData/POP909-Dataset/index.csv')
index

,song_id,name,artist,modify_times,num_beats_per_measure,num_quavers_per_beat,time_signature,time,human_labeled_bars_1,human_labeled_bars_2
0,1,123我爱你,新乐尘符,3,2,2,4/4,193.943960,71,71
1,2,123木头人,黑涩会美眉,4,2,2,4/4,230.475986,58,58
2,3,9420,麦小兜,2,2,2,4/4,228.324284,76,76
3,4,Dear Friend,顺子,2,2,2,4/4,198.945253,56,57
4,5,I Believe,范逸臣,1,2,2,4/4,281.135414,72,71
...,...,...,...,...,...,...,...,...,...,...
904,905,小城故事,邓丽君,5,2,2,4/4,157.476463,47,47
905,906,山不转水转,那英,3,2,2,4/4,195.595774,46,47
906,907,小龙人,小龙人,3,2,2,4/4,124.120583,65,65
907,908,小乌龟,张栋梁,1,2,2,4/4,221.298086,76,76


In [3]:
valid_song_ids = index[index['time_signature'] == '4/4']['song_id'].tolist()
len(valid_song_ids)

857

In [4]:
corpus = dict()
for sid in tqdm(valid_song_ids):
    midi, chords, keys, (structure_1, structure_2) = read_midi(sid)
    corpus['{}_1'.format(sid)] = get_grouped_events(sid, midi, chords, keys, structure_1, token_format='hierarchy')
    corpus['{}_2'.format(sid)] = get_grouped_events(sid, midi, chords, keys, structure_2, token_format='hierarchy')
    
len(corpus)

100%|██████████| 857/857 [01:35<00:00,  8.93it/s]


1714

In [5]:
corpus['2_1']

{'notes': {'MELODY': defaultdict(list,
              {7920: [Note(time=7933, pitch=75, duration=73, velocity=105)],
               8040: [Note(time=8053, pitch=73, duration=61, velocity=66)],
               8160: [Note(time=8173, pitch=75, duration=50, velocity=105)],
               8280: [Note(time=8293, pitch=73, duration=55, velocity=92)],
               8400: [Note(time=8413, pitch=75, duration=193, velocity=111)],
               8640: [Note(time=8653, pitch=71, duration=158, velocity=111)],
               8880: [Note(time=8893, pitch=70, duration=175, velocity=114)],
               9120: [Note(time=9133, pitch=71, duration=61, velocity=108)],
               9240: [Note(time=9253, pitch=73, duration=305, velocity=105)],
               9840: [Note(time=9853, pitch=71, duration=80, velocity=89)],
               9960: [Note(time=9973, pitch=70, duration=57, velocity=87)],
               10080: [Note(time=10093, pitch=71, duration=61, velocity=100)],
               10200: [Note(time=10

In [6]:
with open('./data/hierarchy/corpus.pkl', 'wb') as f:
    pickle.dump(corpus, f)

# Corpus2Tokens(改变bar-text）

In [1]:
from tqdm import tqdm
import random
import pretty_midi
import muspy
import mido

import pandas as pd
import os
import numpy as np
import collections
import pickle
import json

from corpus2tokens import get_event_tokens

In [2]:
with open('./data/hierarchy/corpus.pkl', 'rb') as f:
    corpus = pickle.load(f)
len(corpus)

1714

In [3]:
corpus['629_1']

{'notes': {'MELODY': defaultdict(list,
              {1080: [Note(time=1043, pitch=74, duration=575, velocity=85)],
               1800: [Note(time=1763, pitch=74, duration=193, velocity=104)],
               2040: [Note(time=2003, pitch=72, duration=415, velocity=102)],
               2520: [Note(time=2483, pitch=67, duration=196, velocity=92)],
               2760: [Note(time=2723, pitch=65, duration=176, velocity=98)],
               3000: [Note(time=2963, pitch=67, duration=387, velocity=100)],
               3480: [Note(time=3443, pitch=72, duration=178, velocity=97)],
               3720: [Note(time=3683, pitch=69, duration=584, velocity=97)],
               4440: [Note(time=4403, pitch=67, duration=164, velocity=98)],
               4680: [Note(time=4643, pitch=69, duration=137, velocity=99)],
               4920: [Note(time=4883, pitch=72, duration=289, velocity=100)],
               5160: [Note(time=5123, pitch=74, duration=283, velocity=99)],
               5640: [Note(time=5

In [4]:
tokens = dict()
for key in tqdm(corpus):
    tokens[key] = get_event_tokens(corpus[key])

100%|██████████| 1714/1714 [01:13<00:00, 23.24it/s]


In [5]:
with open('./data/hierarchy/tokens.pkl', 'wb') as f:
    pickle.dump(tokens, f)

In [6]:
len(tokens['629_1'])

1793

In [7]:
tokens['629_1']

[{'type': 'Boundary',
  'bar': '<SOS>',
  'pos': 0,
  'time': 0,
  'tempo': 0,
  'key': 0,
  'structure': 0,
  'chord': 0,
  'track': 0,
  'pitch': 0,
  'duration': 0,
  'velocity': 0},
 {'type': 'Tempo',
  'bar': 1,
  'pos': 'Beat_0',
  'time': 0.0,
  'tempo': 75,
  'key': 0,
  'structure': 0,
  'chord': 0,
  'track': 0,
  'pitch': 0,
  'duration': 0,
  'velocity': 0},
 {'type': 'Structure',
  'bar': '<CONTI>',
  'pos': 'Beat_8',
  'time': 0.006924882629107981,
  'tempo': 0,
  'key': 0,
  'structure': 'A16',
  'chord': 0,
  'track': 0,
  'pitch': 0,
  'duration': 0,
  'velocity': 0},
 {'type': 'Note',
  'bar': '<CONTI>',
  'pos': 'Beat_8',
  'time': -0.13333333333333333,
  'tempo': '<CONTI>',
  'key': '<UNK>',
  'structure': '<CONTI>',
  'chord': '<UNK>',
  'track': 'PIANO',
  'pitch': 58,
  'duration': 600,
  'velocity': 70},
 {'type': 'Note',
  'bar': '<CONTI>',
  'pos': 'Beat_8',
  'time': 0.0,
  'tempo': '<CONTI>',
  'key': '<UNK>',
  'structure': '<CONTI>',
  'chord': '<UNK>',
  

In [8]:
with open('./data/hierarchy/tokens.pkl', 'rb') as f:
    tokens = pickle.load(f)
len(tokens)

1714

In [9]:
with open('./data/base-transformer/tokens.pkl', 'rb') as f:
    old_tokens = pickle.load(f)
len(old_tokens)

1714

In [10]:
def event2df(event):
    keys = list(event[0].keys())

    items_dict = dict()
    for k in keys:
        items_dict[k] = [e[k] for e in event]

    return pd.DataFrame(data=items_dict)

In [11]:
# index = -50
# index += 50
# df[index:index+50]

# # df.head(50)

# df.tail(30)

# df[df['bar'] == 1]



In [12]:
for sid in tqdm(tokens):
    events = tokens[sid]
    old_events = old_tokens[sid]

    bars = sum([t['bar'] for t in events if t['bar'] not in ['<CONTI>', '<SOS>', '<EOS>', 0]])
    old_bars = len([t for t in old_events if t['bar'] == 'Bar'])

    if bars != old_bars:
        print(sid, old_bars, bars)

 11%|█▏        | 197/1714 [00:00<00:01, 920.62it/s]

1_1 73 72
1_2 73 72
16_1 76 75
16_2 76 75
23_1 65 63
23_2 65 63
25_1 91 90
25_2 91 90
28_1 62 61
28_2 62 61
31_1 94 93
31_2 94 93
41_1 96 95
41_2 96 95
46_1 112 105
46_2 112 105
49_1 103 102
49_2 103 102
53_1 74 73
53_2 74 73
54_1 80 79
54_2 80 79
56_1 102 101
56_2 102 101
63_1 95 89
63_2 95 89
65_1 120 119
65_2 120 119
78_1 95 94
78_2 95 94
80_1 84 79
80_2 84 79
88_1 144 140
88_2 144 140
116_1 53 52
116_2 53 52


 23%|██▎       | 402/1714 [00:00<00:01, 966.19it/s]

132_1 72 71
132_2 72 71
135_1 87 86
135_2 87 86
153_1 92 91
153_2 92 91
158_1 104 102
158_2 104 102
163_1 85 84
163_2 85 84
165_1 65 64
165_2 65 64
192_1 72 71
192_2 72 71
194_1 116 108
194_2 116 108
199_1 36 35
199_2 36 35
207_1 127 126
207_2 127 126
208_1 141 139
208_2 141 139
212_1 100 99
212_2 100 99
224_1 85 81
224_2 85 81
225_1 98 96
225_2 98 96
229_1 75 71
229_2 75 71
236_1 137 133
236_2 137 133
245_1 84 83
245_2 84 83
249_1 111 110
249_2 111 110
251_1 111 107
251_2 111 107
255_1 87 85
255_2 87 85


 36%|███▋      | 622/1714 [00:00<00:01, 1029.83it/s]

...

 83%|████████▎ | 1416/1714 [00:01<00:00, 1095.22it/s]

660_1 102 101
660_2 102 101
667_1 135 132
667_2 135 132
676_1 71 70
676_2 71 70
680_1 81 80
680_2 81 80
691_1 115 114
691_2 115 114
725_1 94 93
725_2 94 93
736_1 77 76
736_2 77 76
748_1 75 73
748_2 75 73
754_1 122 121
754_2 122 121
757_1 85 84
757_2 85 84
762_1 100 99
762_2 100 99
767_1 80 79
767_2 80 79
779_1 120 117
779_2 120 117
785_1 109 108
785_2 109 108
786_1 64 59
786_2 64 59
787_1 135 133
787_2 135 133
791_1 74 69
791_2 74 69
792_1 85 84
792_2 85 84


 95%|█████████▌| 1635/1714 [00:01<00:00, 1026.65it/s]

797_1 59 57
797_2 59 57
800_1 123 114
800_2 123 114
807_1 106 102
807_2 106 102
828_1 64 63
828_2 64 63
830_1 129 128
830_2 129 128
842_1 109 108
842_2 109 108
847_1 83 82
847_2 83 82
861_1 136 135
861_2 136 135
900_1 72 69
900_2 72 69
901_1 79 78
901_2 79 78
903_1 88 87
903_2 88 87
905_1 52 48
905_2 52 48


100%|██████████| 1714/1714 [00:01<00:00, 1050.37it/s]


In [13]:
sid = '845_1'
df = event2df(tokens[sid])
old_df = event2df(old_tokens[sid])
len(df), len(old_df)

(2460, 2463)

In [14]:
df[~df['bar'].isin([0, '<CONTI>', 1])]

,type,bar,pos,time,tempo,key,structure,chord,track,pitch,duration,velocity
0,Boundary,<SOS>,0,0.00,0,0,0,0,0,0,0,0
2239,Chord,3,Beat_1,-0.15,<CONTI>,<UNK>,<CONTI>,D:sus2,0,0,0,0
2459,Boundary,<EOS>,0,0.00,0,0,0,0,0,0,0,0


In [15]:
old_df[2230:2250]

,type,bar,pos,time,tempo,key,structure,chord,track,pitch,duration,velocity
2230,Note,<CONTI>,Beat_13,-0.150000,<CONTI>,<CONTI>,<CONTI>,<CONTI>,BRIDGE,74,360,80
2231,Note,<CONTI>,Beat_13,-0.150000,<CONTI>,<CONTI>,<CONTI>,<CONTI>,PIANO,55,120,76
2232,Note,<CONTI>,Beat_13,-0.150000,<CONTI>,<CONTI>,<CONTI>,<CONTI>,PIANO,62,360,80
2233,Note,<CONTI>,Beat_15,-0.150000,<CONTI>,<CONTI>,<CONTI>,<CONTI>,PIANO,55,120,82
2234,Note,Bar,Beat_1,-0.150000,<CONTI>,<CONTI>,<CONTI>,<CONTI>,BRIDGE,72,2040,72
2235,Note,<CONTI>,Beat_1,-0.150000,<CONTI>,<CONTI>,<CONTI>,<CONTI>,BRIDGE,76,2040,74
2236,Note,<CONTI>,Beat_1,-0.150000,<CONTI>,<CONTI>,<CONTI>,<CONTI>,PIANO,48,960,86
2237,Note,<CONTI>,Beat_1,-0.150000,<CONTI>,<CONTI>,<CONTI>,<CONTI>,PIANO,55,960,80
2238,Note,<CONTI>,Beat_1,-0.150000,<CONTI>,<CONTI>,<CONTI>,<CONTI>,PIANO,60,960,82
2239,Structure,Bar,Beat_1,0.871981,0,0,x11,0,0,0,0,0


# 由tokens导出midi

## Preview

In [16]:
import random
import json
import pickle
from tqdm import tqdm
import pandas as pd
from corpus2tokens import write_midi

In [17]:
with open('./data/hierarchy/tokens.pkl', 'rb') as f:
    tokens = pickle.load(f)
len(tokens)

1714

In [18]:
def event2df(event):
    keys = list(event[0].keys())

    items_dict = dict()
    for k in keys:
        items_dict[k] = [e[k] for e in event]

    return pd.DataFrame(data=items_dict)

In [19]:
sids = list(tokens.keys())
sid = random.sample(sids, 1)[0]
df = event2df(tokens[sid])
sid, len(df)

('305_1', 1515)

In [20]:
index = -50
index += 50

df[index:index+50]

,type,bar,pos,time,tempo,key,structure,chord,track,pitch,duration,velocity
0,Boundary,<SOS>,0,0.000000,0,0,0,0,0,0,0,0
1,Tempo,1,Beat_0,0.000000,76,0,0,0,0,0,0,0
2,Chord,<CONTI>,Beat_10,-0.100000,<CONTI>,<UNK>,<UNK>,Ab:maj,0,0,0,0
3,Structure,<CONTI>,Beat_14,0.009872,0,0,i3,0,0,0,0,0
4,Note,<CONTI>,Beat_14,-0.100000,<CONTI>,<UNK>,<CONTI>,<CONTI>,BRIDGE,72,240,116
5,Note,<CONTI>,Beat_14,-0.100000,<CONTI>,<UNK>,<CONTI>,<CONTI>,PIANO,44,1800,86
6,Note,<CONTI>,Beat_14,-0.100000,<CONTI>,<UNK>,<CONTI>,<CONTI>,PIANO,56,1920,88
7,Note,1,Beat_0,-0.141667,<CONTI>,<UNK>,<CONTI>,<CONTI>,PIANO,63,840,88
8,Note,<CONTI>,Beat_2,-0.108333,<CONTI>,<UNK>,<CONTI>,<CONTI>,PIANO,72,240,88
9,Note,<CONTI>,Beat_8,-0.166667,<CONTI>,<UNK>,<CONTI>,<CONTI>,PIANO,72,240,100


In [21]:
df[df['type'] == 'Structure']

,type,bar,pos,time,tempo,key,structure,chord,track,pitch,duration,velocity
3,Structure,<CONTI>,Beat_14,0.009872,0,0,i3,0,0,0,0,0
28,Structure,<CONTI>,Beat_14,0.043963,0,0,A8,0,0,0,0,0
132,Structure,<CONTI>,Beat_14,0.134872,0,0,A8,0,0,0,0,0
238,Structure,<CONTI>,Beat_14,0.225781,0,0,B8,0,0,0,0,0
434,Structure,<CONTI>,Beat_14,0.316690,0,0,B8,0,0,0,0,0
616,Structure,<CONTI>,Beat_14,0.407599,0,0,X2,0,0,0,0,0
644,Structure,<CONTI>,Beat_14,0.430327,0,0,A8,0,0,0,0,0
736,Structure,<CONTI>,Beat_14,0.521236,0,0,A8,0,0,0,0,0
842,Structure,<CONTI>,Beat_14,0.612145,0,0,B8,0,0,0,0,0
1038,Structure,<CONTI>,Beat_14,0.703054,0,0,B8,0,0,0,0,0


## 导出

In [22]:
tokens['1_2']

[{'type': 'Boundary',
  'bar': '<SOS>',
  'pos': 0,
  'time': 0,
  'tempo': 0,
  'key': 0,
  'structure': 0,
  'chord': 0,
  'track': 0,
  'pitch': 0,
  'duration': 0,
  'velocity': 0},
 {'type': 'Tempo',
  'bar': 1,
  'pos': 'Beat_0',
  'time': 0.0,
  'tempo': 90,
  'key': 0,
  'structure': 0,
  'chord': 0,
  'track': 0,
  'pitch': 0,
  'duration': 0,
  'velocity': 0},
 {'type': 'Structure',
  'bar': '<CONTI>',
  'pos': 'Beat_0',
  'time': 0.00028935185185185184,
  'tempo': 0,
  'key': 0,
  'structure': 'i4',
  'chord': 0,
  'track': 0,
  'pitch': 0,
  'duration': 0,
  'velocity': 0},
 {'type': 'Chord',
  'bar': '<CONTI>',
  'pos': 'Beat_0',
  'time': 0.3333333333333333,
  'tempo': '<CONTI>',
  'key': '<UNK>',
  'structure': '<CONTI>',
  'chord': 'B:maj',
  'track': 0,
  'pitch': 0,
  'duration': 0,
  'velocity': 0},
 {'type': 'Note',
  'bar': '<CONTI>',
  'pos': 'Beat_0',
  'time': 0.3333333333333333,
  'tempo': '<CONTI>',
  'key': '<UNK>',
  'structure': '<CONTI>',
  'chord': '<CONT

In [23]:
for sid, words in tqdm(tokens.items()):
    write_midi(words, './data/hierarchy/midi/{}.mid'.format(sid))

100%|██████████| 1714/1714 [02:26<00:00, 11.71it/s]


# Parse Hierarchical Data

In [24]:
import random
import json
import pickle
from tqdm import tqdm
import pandas as pd
from tokens2tree import *

In [25]:
with open('./data/hierarchy/tokens.pkl', 'rb') as f:
    tokens_dict = pickle.load(f)
len(tokens_dict)

1714

In [26]:
def event2df(event):
    keys = list(event[0].keys())

    items_dict = dict()
    for k in keys:
        items_dict[k] = [e[k] for e in event]

    return pd.DataFrame(data=items_dict)

In [27]:
# df = event2df(tokens_dict['209_1'])

# df.head(20)

In [28]:
for sid in tqdm(tokens_dict):
    tokens = delete_performance_tokens(tokens_dict[sid])
    tokens_dict[sid] = refine_structure(tokens)

100%|██████████| 1714/1714 [00:04<00:00, 426.73it/s]


In [29]:
sids = list(tokens_dict.keys())
sid = random.sample(sids, 1)[0]
df = event2df(tokens_dict[sid])
print(sid, len(df))

assert len(df[df['structure']=='<UNK>']) == 0

df.head(50)

312_2 1872


,type,bar,pos,time,tempo,structure,chord,track,pitch,duration
0,Boundary,<SOS>,0,0.000000,0,0,0,0,0,0
1,Tempo,1,Beat_0,0.000000,98,0,0,0,0,0
2,Structure,<CONTI>,Beat_5,0.003396,0,i,0,0,0,0
3,Chord,<CONTI>,Beat_5,0.325000,<CONTI>,<CONTI>,C:maj,0,0,0
4,Note,<CONTI>,Beat_5,0.325000,<CONTI>,<CONTI>,<CONTI>,BRIDGE,67,720
5,Note,<CONTI>,Beat_5,0.325000,<CONTI>,<CONTI>,<CONTI>,PIANO,48,600
6,Note,<CONTI>,Beat_7,0.325000,<CONTI>,<CONTI>,<CONTI>,PIANO,55,600
7,Note,<CONTI>,Beat_9,0.325000,<CONTI>,<CONTI>,<CONTI>,PIANO,60,360
8,Note,<CONTI>,Beat_11,0.325000,<CONTI>,<CONTI>,<CONTI>,BRIDGE,72,240
9,Note,<CONTI>,Beat_11,0.325000,<CONTI>,<CONTI>,<CONTI>,PIANO,64,240


In [30]:
df[df['type']=='Structure']

,type,bar,pos,time,tempo,structure,chord,track,pitch,duration
2,Structure,<CONTI>,Beat_5,0.003396,0,i,0,0,0,0
90,Structure,<CONTI>,Beat_5,0.074825,0,A,0,0,0,0
198,Structure,<CONTI>,Beat_5,0.156457,0,A,0,0,0,0
333,Structure,<CONTI>,Beat_5,0.238090,0,B,0,0,0,0
526,Structure,<CONTI>,Beat_5,0.319723,0,B,0,0,0,0
713,Structure,<CONTI>,Beat_5,0.401355,0,x,0,0,0,0
773,Structure,<CONTI>,Beat_5,0.442172,0,A,0,0,0,0
907,Structure,<CONTI>,Beat_5,0.523804,0,B,0,0,0,0
1100,Structure,<CONTI>,Beat_5,0.605437,0,B,0,0,0,0
1292,Structure,<CONTI>,Beat_5,0.687070,0,x,0,0,0,0


In [31]:
tree_tokens_dict = dict()
tree_tokens_new_created = dict()
for sid in tqdm(tokens_dict):
    tree_tokens, new_created_num = parse_flatten_tokens(tokens_dict[sid])
    tree_tokens_dict[sid] = tree_tokens
    tree_tokens_new_created[sid] = new_created_num

100%|██████████| 1714/1714 [00:01<00:00, 958.81it/s] 


In [32]:
# sid = '487_1'
# df = event2df(tokens[sid])
# index = 626
# df[max(0, index-10):index+30]

In [33]:
for sid in tqdm(tokens_dict):
    df = event2df(tokens_dict[sid])
    gt_structure_seq_df = df[df['structure'] != '<CONTI>']
    gt_chord_seq_df = df[(df['structure'] != '<CONTI>')
                         | (df['chord'] != '<CONTI>')]

    tree_tokens = tree_tokens_dict[sid]
    tree_types = []
    for t in tree_tokens:
        if type(t) != list:
            tree_types.append(t['type'])
        else:
            tt_types = []
            for tt in t:
                if type(tt) != list:
                    tt_types.append(tt['type'])
                else:
                    #                     tt_types += [ttt['type'] for ttt in tt]
                    tt_types.append(tt[0]['type'])
            tree_types.append(tt_types)
    tree_tokens = tree_types

    if len(tree_tokens) != len(gt_structure_seq_df):
        print('Seq', sid, len(tree_tokens), len(gt_structure_seq_df))
        break

    chord_seq_num = 0
    for t in tree_tokens:
        if type(t) != list:
            chord_seq_num += 1
        else:
            chord_seq_num += len(t)

    if chord_seq_num != len(gt_chord_seq_df) + tree_tokens_new_created[sid]:
        print('Chord', sid, chord_seq_num, len(gt_chord_seq_df),
              'sub = {}'.format(chord_seq_num - len(gt_chord_seq_df)))
#         break

100%|██████████| 1714/1714 [00:09<00:00, 178.08it/s]


In [34]:
with open('./data/hierarchy/tree_tokens.pkl', 'wb') as f:
    pickle.dump(tree_tokens_dict, f)

In [35]:
len(gt_structure_seq_df), len(gt_chord_seq_df)

(13, 64)

In [36]:
gt_structure_seq_df

,type,bar,pos,time,tempo,structure,chord,track,pitch,duration
0,Boundary,<SOS>,0,0.000000,0,0,0,0,0,0
1,Tempo,1,Beat_0,0.000000,104,0,0,0,0,0
2,Structure,<CONTI>,Beat_10,0.014159,0,i,0,0,0,0
138,Structure,<CONTI>,Beat_10,0.130438,0,A,0,0,0,0
233,Structure,<CONTI>,Beat_10,0.223462,0,A,0,0,0,0
335,Structure,<CONTI>,Beat_10,0.316485,0,B,0,0,0,0
426,Structure,<CONTI>,Beat_10,0.409508,0,A,0,0,0,0
540,Structure,<CONTI>,Beat_10,0.502531,0,x,0,0,0,0
676,Structure,<CONTI>,Beat_10,0.618811,0,A,0,0,0,0
771,Structure,<CONTI>,Beat_10,0.711834,0,A,0,0,0,0


In [37]:
gt_chord_seq_df

,type,bar,pos,time,tempo,structure,chord,track,pitch,duration
0,Boundary,<SOS>,0,0.000000,0,0,0,0,0,0
1,Tempo,1,Beat_0,0.000000,104,0,0,0,0,0
2,Structure,<CONTI>,Beat_10,0.014159,0,i,0,0,0,0
3,Chord,<CONTI>,Beat_10,-0.258333,<CONTI>,<CONTI>,C:maj/5,0,0,0
35,Chord,<CONTI>,Beat_10,-0.258333,<CONTI>,<CONTI>,G:maj,0,0,0
...,...,...,...,...,...,...,...,...,...,...
991,Chord,<CONTI>,Beat_10,-0.258333,<CONTI>,<CONTI>,C:maj/5,0,0,0
1023,Chord,<CONTI>,Beat_10,-0.258333,<CONTI>,<CONTI>,G:maj,0,0,0
1052,Chord,<CONTI>,Beat_10,-0.258333,<CONTI>,<CONTI>,C:maj/5,0,0,0
1073,Chord,<CONTI>,Beat_2,-0.258333,<CONTI>,<CONTI>,C:maj,0,0,0


# Statistics

In [1]:
import pickle
import random
import json
from tqdm import tqdm
import pandas as pd

In [2]:
with open('./data/hierarchy/tree_tokens.pkl', 'rb') as f:
    tree_tokens_dict = pickle.load(f)
    
len(tree_tokens_dict)

1714

In [3]:
with open('./data/hierarchy/tokens.pkl', 'rb') as f:
    tokens_dict = pickle.load(f)
len(tokens_dict)

1714

In [4]:
def event2df(event):
    keys = list(event[0].keys())

    items_dict = dict()
    for k in keys:
        items_dict[k] = [e[k] for e in event]

    return pd.DataFrame(data=items_dict)

## Structure算作一个节点

### max_seq

In [5]:
structure_seq_num = [len(tokens) for tokens in tree_tokens_dict.values()]
len(structure_seq_num)

1714

In [6]:
structure_df = pd.DataFrame(
    {'sid': list(tree_tokens_dict.keys()), 'num': structure_seq_num})

In [7]:
structure_df.describe()

,num
count,1714.000000
mean,15.987165
std,3.672027
min,5.000000
25%,13.000000
50%,16.000000
75%,18.000000
max,42.000000


In [8]:
structure_df.sort_values('num')

,sid,num
187,98_2,5
186,98_1,5
376,199_1,6
843,447_2,6
842,447_1,7
...,...,...
1036,545_1,31
1564,830_1,32
1217,641_2,33
1216,641_1,33


In [9]:
# df = event2df(tokens_dict['196_1'])

# df

# df[df['structure'] != '<CONTI>']

### 对于一个structure的节点内部：max_chord

In [10]:
chord_sid = []
chord_num = []

for sid, tokens in tree_tokens_dict.items():
    for i, t in enumerate(tokens):
        if type(t) != list:
            continue
        
        chord_sid.append('{}_structure_{}'.format(sid, i))
        chord_num.append(len(t))
        
len(chord_num), len(chord_sid)

(22260, 22260)

In [11]:
chord_df = pd.DataFrame({'id': chord_sid, 'num': chord_num})

In [12]:
chord_df.describe()

,num
count,22260.000000
mean,11.634501
std,6.081888
min,1.000000
25%,8.000000
50%,10.000000
75%,15.000000
max,104.000000


In [13]:
chord_df.sort_values('num')

,id,num
7530,310_2_structure_11,1
5118,213_2_structure_8,1
6762,278_1_structure_4,1
20689,845_2_structure_24,1
7514,310_1_structure_11,1
...,...,...
4318,184_2_structure_8,94
6785,278_2_structure_13,99
6772,278_1_structure_14,99
4312,184_2_structure_2,104


In [14]:
tree_tokens_dict['184_1'][2]

[{'type': 'Structure',
  'bar': '<CONTI>',
  'pos': 'Beat_3',
  'time': 0.005184294871794871,
  'tempo': 0,
  'structure': 'i',
  'chord': 0,
  'track': 0,
  'pitch': 0,
  'duration': 0},
 [{'type': 'Chord',
   'bar': '<CONTI>',
   'pos': 'Beat_3',
   'time': 0.39166666666666666,
   'tempo': '<CONTI>',
   'structure': '<CONTI>',
   'chord': 'Eb:maj/3',
   'track': 0,
   'pitch': 0,
   'duration': 0},
  {'type': 'Note',
   'bar': '<CONTI>',
   'pos': 'Beat_5',
   'time': 0.39166666666666666,
   'tempo': '<CONTI>',
   'structure': '<CONTI>',
   'chord': '<CONTI>',
   'track': 'PIANO',
   'pitch': 67,
   'duration': 120},
  {'type': 'Note',
   'bar': '<CONTI>',
   'pos': 'Beat_6',
   'time': 0.39166666666666666,
   'tempo': '<CONTI>',
   'structure': '<CONTI>',
   'chord': '<CONTI>',
   'track': 'PIANO',
   'pitch': 70,
   'duration': 120}],
 [{'type': 'Chord',
   'bar': '<CONTI>',
   'pos': 'Beat_7',
   'time': 0.39166666666666666,
   'tempo': '<CONTI>',
   'structure': '<CONTI>',
   'ch

In [15]:
index = -50
index += 50

df = event2df(tokens_dict['184_1'])
df[(df['structure'] != '<CONTI>') | (df['chord'] != '<CONTI>')][index:index+50]

,type,bar,pos,time,tempo,key,structure,chord,track,pitch,duration,velocity
0,Boundary,<SOS>,0,0.000000,0,0,0,0,0,0,0,0
1,Tempo,1,Beat_0,0.000000,57,0,0,0,0,0,0,0
2,Chord,<CONTI>,Beat_3,0.391667,<CONTI>,<UNK>,<UNK>,Eb:maj/3,0,0,0,0
3,Structure,<CONTI>,Beat_5,0.005184,0,0,i8,0,0,0,0,0
6,Chord,<CONTI>,Beat_7,0.391667,<CONTI>,<UNK>,<CONTI>,C:min,0,0,0,0
19,Chord,<CONTI>,Beat_15,0.391667,<CONTI>,<UNK>,<CONTI>,F:min7,0,0,0,0
36,Chord,<CONTI>,Beat_7,0.391667,<CONTI>,<UNK>,<CONTI>,Bb:maj,0,0,0,0
50,Chord,<CONTI>,Beat_15,0.391667,<CONTI>,<UNK>,<CONTI>,C:sus2,0,0,0,0
59,Chord,<CONTI>,Beat_3,0.391667,<CONTI>,<UNK>,<CONTI>,C:min/5,0,0,0,0
72,Chord,<CONTI>,Beat_7,0.391667,<CONTI>,<UNK>,<CONTI>,F:sus2,0,0,0,0


### 对于一个chord的节点内部：max_note

In [16]:
note_sid = []
note_num = []

for sid, tokens in tree_tokens_dict.items():
    for i, t in enumerate(tokens):
        if type(t) != list:
            continue
        
        for j, tt in enumerate(t):
            if type(tt) != list:
                continue
                
            note_sid.append('{}_structure_{}_chord_{}'.format(sid, i, j))
            note_num.append(len(tt))
            
len(note_num), len(note_sid)

(231469, 231469)

In [17]:
note_df = pd.DataFrame({'id': note_sid, 'num': note_num})

In [18]:
note_df.describe()

,num
count,231469.000000
mean,13.513175
std,6.872476
min,1.000000
25%,9.000000
50%,13.000000
75%,17.000000
max,129.000000


In [19]:
note_df.sort_values('num')

,id,num
27366,108_2_structure_8_chord_19,1
161254,623_2_structure_3_chord_5,1
105500,404_1_structure_15_chord_6,1
112258,431_1_structure_2_chord_3,1
98300,373_2_structure_15_chord_5,1
...,...,...
25805,100_2_structure_10_chord_2,81
24701,95_2_structure_3_chord_34,103
24506,95_1_structure_3_chord_34,103
25729,100_1_structure_10_chord_8,129


In [20]:
df = event2df(tree_tokens_dict['100_2'][13][8])

df

,type,bar,pos,time,tempo,structure,chord,track,pitch,duration
0,Chord,<CONTI>,Beat_14,-0.158333,<CONTI>,<CONTI>,B:maj,0,0,0
1,Note,<CONTI>,Beat_14,-0.291667,<CONTI>,<CONTI>,<CONTI>,BRIDGE,57,0
2,Note,<CONTI>,Beat_14,-0.233333,<CONTI>,<CONTI>,<CONTI>,PIANO,59,0
3,Note,<CONTI>,Beat_14,0.075000,<CONTI>,<CONTI>,<CONTI>,PIANO,71,0
4,Note,<CONTI>,Beat_14,0.100000,<CONTI>,<CONTI>,<CONTI>,BRIDGE,59,0
...,...,...,...,...,...,...,...,...,...,...
124,Note,<CONTI>,Beat_13,-0.233333,<CONTI>,<CONTI>,<CONTI>,BRIDGE,74,0
125,Note,<CONTI>,Beat_13,-0.025000,<CONTI>,<CONTI>,<CONTI>,PIANO,59,0
126,Note,<CONTI>,Beat_13,0.058333,<CONTI>,<CONTI>,<CONTI>,BRIDGE,76,0
127,Note,<CONTI>,Beat_13,0.258333,<CONTI>,<CONTI>,<CONTI>,PIANO,71,0


In [21]:
df[0:50]
# df[50:100]
# df[100:]

,type,bar,pos,time,tempo,structure,chord,track,pitch,duration
0,Chord,<CONTI>,Beat_14,-0.158333,<CONTI>,<CONTI>,B:maj,0,0,0
1,Note,<CONTI>,Beat_14,-0.291667,<CONTI>,<CONTI>,<CONTI>,BRIDGE,57,0
2,Note,<CONTI>,Beat_14,-0.233333,<CONTI>,<CONTI>,<CONTI>,PIANO,59,0
3,Note,<CONTI>,Beat_14,0.075000,<CONTI>,<CONTI>,<CONTI>,PIANO,71,0
4,Note,<CONTI>,Beat_14,0.100000,<CONTI>,<CONTI>,<CONTI>,BRIDGE,59,0
5,Note,<CONTI>,Beat_14,0.316667,<CONTI>,<CONTI>,<CONTI>,BRIDGE,62,0
6,Note,<CONTI>,Beat_14,0.350000,<CONTI>,<CONTI>,<CONTI>,PIANO,59,0
7,Note,<CONTI>,Beat_15,-0.416667,<CONTI>,<CONTI>,<CONTI>,BRIDGE,64,0
8,Note,<CONTI>,Beat_15,-0.325000,<CONTI>,<CONTI>,<CONTI>,PIANO,71,0
9,Note,<CONTI>,Beat_15,-0.208333,<CONTI>,<CONTI>,<CONTI>,BRIDGE,66,0


In [31]:
# df = event2df(tokens_dict['100_2'])
# df[df['structure'] != '<CONTI>'][:15]

# # index = 1619
# index += 50

# df[index:index+50]

### 从song的维度

In [32]:
len(tree_tokens_dict)

1714

In [33]:
song_ids = []
seq_num = []
max_chord_num = []
max_note_num = []

for sid, tokens in tree_tokens_dict.items():
    song_ids.append(sid)
    seq_num.append(len(tokens))

    chord_num = []
    note_num = []

    for t in tokens:
        if type(t) != list:
            chord_num.append(1)
        else:
            chord_num.append(len(t))

            for tt in t:
                if type(tt) != list:
                    note_num.append(1)
                else:
                    note_num.append(len(tt))

    max_chord_num.append(max(chord_num))
    max_note_num.append(max(note_num))

df = pd.DataFrame({'sid': song_ids, 'seq_num': seq_num,
                   'max_chord_num': max_chord_num, 'max_note_num': max_note_num})
len(df)

1714

In [34]:
df

,sid,seq_num,max_chord_num,max_note_num
0,1_1,19,15,36
1,1_2,21,11,36
2,2_1,14,19,27
3,2_2,14,22,27
4,3_1,18,12,34
...,...,...,...,...
1709,907_2,15,17,26
1710,908_1,13,22,35
1711,908_2,13,22,35
1712,909_1,13,9,34


In [35]:
df.describe()

,seq_num,max_chord_num,max_note_num
count,1714.000000,1714.000000,1714.000000
mean,15.987165,20.572345,33.188448
std,3.672027,9.177803,10.649874
min,5.000000,6.000000,12.000000
25%,13.000000,16.000000,26.000000
50%,16.000000,19.000000,31.000000
75%,18.000000,23.000000,38.000000
max,42.000000,104.000000,129.000000


In [39]:
df[(df['max_chord_num'] < 30) &  (df['max_note_num'] < 40)]

,sid,seq_num,max_chord_num,max_note_num
0,1_1,19,15,36
1,1_2,21,11,36
2,2_1,14,19,27
3,2_2,14,22,27
4,3_1,18,12,34
...,...,...,...,...
1709,907_2,15,17,26
1710,908_1,13,22,35
1711,908_2,13,22,35
1712,909_1,13,9,34


## Chord 算作一个节点

### max_seq

In [40]:
len(tree_tokens_dict)

1714

In [44]:
chord_sid = []
chord_seq_num = []

for sid, tokens in tree_tokens_dict.items():
    seq_num = sum([1 if type(t) != list else len(t) for t in tokens])
    chord_seq_num.append(seq_num)
    chord_sid.append(sid)

chord_seq_df = pd.DataFrame({'sid': chord_sid, 'num': chord_seq_num})

In [45]:
chord_seq_df.describe()

,num
count,1714.000000
mean,154.099183
std,42.984263
min,25.000000
25%,125.000000
50%,147.000000
75%,176.000000
max,439.000000


In [46]:
chord_seq_df.sort_values('num')

,sid,num
187,98_2,25
186,98_1,25
1443,764_2,55
1442,764_1,55
843,447_2,56
...,...,...
81,43_2,313
346,184_1,317
347,184_2,321
371,196_2,420


In [48]:
sid = '196_1'
df = event2df(tokens_dict[sid])
df[df['chord'] != '<CONTI>']

,type,bar,pos,time,tempo,key,structure,chord,track,pitch,duration,velocity
0,Boundary,<SOS>,0,0.00000,0,0,0,0,0,0,0,0
1,Tempo,1,Beat_0,0.00000,100,0,0,0,0,0,0,0
2,Chord,<CONTI>,Beat_7,0.00000,<CONTI>,<UNK>,<UNK>,Eb:min,0,0,0,0
3,Structure,<CONTI>,Beat_11,0.00349,0,0,i8,0,0,0,0,0
11,Chord,<CONTI>,Beat_15,0.00000,<CONTI>,<UNK>,<CONTI>,Eb:min,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4002,Chord,<CONTI>,Beat_7,0.00000,<CONTI>,<UNK>,<CONTI>,Eb:min/b3,0,0,0,0
4007,Chord,<CONTI>,Beat_11,0.00000,<CONTI>,<UNK>,<CONTI>,Ab:min,0,0,0,0
4019,Chord,<CONTI>,Beat_3,0.00000,<CONTI>,<UNK>,<CONTI>,B:maj7/7,0,0,0,0
4024,Chord,<CONTI>,Beat_7,0.00000,<CONTI>,<UNK>,<CONTI>,Ab:sus2,0,0,0,0


### 对于一个chord的节点内部：max_note

In [49]:
note_sid = []
note_num = []

for sid, tokens in tree_tokens_dict.items():
    for i, t in enumerate(tokens):
        if type(t) != list:
            continue
        
        for j, tt in enumerate(t):
            if type(tt) != list:
                continue
                
            note_sid.append('{}_structure_{}_chord_{}'.format(sid, i, j))
            note_num.append(len(tt))
            
len(note_num), len(note_sid)

(231469, 231469)

In [50]:
note_df = pd.DataFrame({'id': note_sid, 'num': note_num})

In [51]:
note_df.describe()

,num
count,231469.000000
mean,13.513175
std,6.872476
min,1.000000
25%,9.000000
50%,13.000000
75%,17.000000
max,129.000000


In [52]:
note_df.sort_values('num')

,id,num
27366,108_2_structure_8_chord_19,1
161254,623_2_structure_3_chord_5,1
105500,404_1_structure_15_chord_6,1
112258,431_1_structure_2_chord_3,1
98300,373_2_structure_15_chord_5,1
...,...,...
25805,100_2_structure_10_chord_2,81
24701,95_2_structure_3_chord_34,103
24506,95_1_structure_3_chord_34,103
25729,100_1_structure_10_chord_8,129


### 从song的维度

In [5]:
len(tree_tokens_dict)

1714

In [6]:
song_ids = []
seq_num = []
max_note_num = []

for sid, tokens in tree_tokens_dict.items():
    song_ids.append(sid)
    seq_num.append(sum([1 if type(t) != list else len(t) for t in tokens]))

    note_num = []

    for t in tokens:
        if type(t) != list:
            continue
        else:
            for tt in t:
                if type(tt) != list:
                    note_num.append(1)
                else:
                    note_num.append(len(tt))

    max_note_num.append(max(note_num))

df = pd.DataFrame({'sid': song_ids, 'seq_num': seq_num, 'max_note_num': max_note_num})
len(df)

1714

In [7]:
df

,sid,seq_num,max_note_num
0,1_1,170,36
1,1_2,172,36
2,2_1,127,27
3,2_2,127,27
4,3_1,113,34
...,...,...,...
1709,907_2,129,26
1710,908_1,126,35
1711,908_2,126,35
1712,909_1,64,34


In [8]:
df.describe()

,seq_num,max_note_num
count,1714.000000,1714.000000
mean,154.099183,33.188448
std,42.984263,10.649874
min,25.000000,12.000000
25%,125.000000,26.000000
50%,147.000000,31.000000
75%,176.000000,38.000000
max,439.000000,129.000000


In [12]:
df[(df['seq_num'] < 256) & (df['max_note_num'] < 40)]

,sid,seq_num,max_note_num
0,1_1,170,36
1,1_2,172,36
2,2_1,127,27
3,2_2,127,27
4,3_1,113,34
...,...,...,...
1709,907_2,129,26
1710,908_1,126,35
1711,908_2,126,35
1712,909_1,64,34
